In [1]:
# Scientific libraries		              
import numpy as np		              
import scipy as sp		              
import pandas as pd		              
import json		              
import missingno as msno		              

# Loading Plotting Modules		              
import matplotlib		              
import matplotlib.pyplot as plt		              
import seaborn as sns		              
%matplotlib inline		              
import chart_studio.plotly as py		              
import plotly.figure_factory as ff		              
import plotly.graph_objects as go		              
import plotly.express as px		              
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot		              
init_notebook_mode(connected=True)		              
		              
import os		              
import gc

In [2]:
# Setting Data Frame Options                      
pd.set_option('display.max_rows', 40)                      
pd.set_option('display.max_columns', 50)                      
pd.set_option('display.width', 100)                      
pd.set_option('display.max_colwidth', 100)                      

# Setting Plot Configuration                      
sns.set(rc={'figure.figsize':(19,11)}, style = 'white')

In [3]:
test = pd.read_csv('./outputs/imputed_test_mf.csv')                
m, n = test.shape
print('Data Frame: {} x {}'.format(m,n))                
test.head()

Data Frame: 890 x 79


,galactic year,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",...,"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,Gender Inequality Index (GII),galaxy,Private galaxy capital flows (% of GGP),"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total"
0,1007012.0,0.868360,72.982208,19198.499040,0.700413,16.145285,11.113581,0.719579,0.825915,181.824530,76.357958,71.796137,71.977811,819.245976,72.269765,113.394831,75.983899,12.851245,426.589757,59.793499,73.206969,79.051548,80.144699,87.255770,77.624057,...,NaN,NaN,102.306960,NaN,NaN,NaN,75.084941,16.383957,NaN,92046.182973,129.047095,578.672958,70.150003,39.261197,11.067889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andromeda Galaxy (M31),32.163414,NaN
1,1007012.0,0.772696,73.590698,18345.350181,0.684405,12.167872,7.083681,0.650826,0.505962,193.187549,79.263683,71.723281,37.085742,849.398477,53.465398,139.090298,88.145153,14.152255,496.779346,57.583581,68.021732,74.795340,73.158109,89.776748,124.050371,...,19268.820517,12931.719986,99.189863,77.742674,54.263456,69.496155,38.331603,9.435519,62.266114,92843.780990,131.408452,429.753270,104.910863,34.341097,6.029590,0.673039,0.714560,0.976994,147.451303,164.334975,6.673241,0.687616,Andromeda I,30.668839,5.784397
2,1007012.0,0.941941,81.973436,26708.317145,0.925563,18.175905,13.909442,0.960263,0.937407,74.678611,122.988241,136.685519,34.865297,712.914221,93.116852,59.810528,34.960991,31.242181,435.976780,54.971127,78.814709,87.318457,54.861556,51.049333,72.544281,...,46130.422627,29804.266408,133.499902,112.410524,110.028144,112.483100,29.294321,9.716045,124.671861,98471.178149,117.291927,216.424101,152.699066,89.310138,10.114774,0.975232,0.949269,1.078944,91.321283,80.725039,16.251653,0.467704,Andromeda III,25.648367,3.641324
3,1007012.0,1.079857,89.351427,54738.000742,1.016415,19.927563,12.917355,1.085718,1.010280,65.576254,113.617318,144.223620,27.907457,994.869334,93.992349,50.614834,37.408702,39.063774,452.063178,68.716581,87.959292,90.227666,83.641251,43.62177

In [144]:
# RMSE Train: 0.001800391617117943
# RMSE Valid: 0.052504038023260875

preds = pd.read_csv('../modeling/outputs/y_pred_lgbm_mf_0525val.csv')
preds['opt_pred'] = 0
preds['eei'] = test['existence expectancy index']
preds['p_increase'] = -np.log(preds['y_pred'] + 0.01) + 3
preds['p_increase2'] = preds['p_increase'] ** 2
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2
0,0,0.041248,0,0.868360,5.971077,35.653766
1,1,0.046107,0,0.772696,5.880488,34.580138
2,2,0.114819,0,0.941941,5.080892,25.815461
3,3,0.177866,0,1.079857,4.672029,21.827850
4,4,0.224418,0,1.079557,4.450649,19.808277
...,...,...,...,...,...,...
885,885,0.040630,0,0.850609,5.983221,35.798930
886,886,0.053313,0,0.808027,5.759666,33.173747
887,887,0.062798,0,1.012209,5.620070,31.585187
888,888,0.040166,0,0.704373,5.992420,35.909100


### Optimization

We will try to maximize sum of likely increase index for all galaxies, which for one galaxy is described as follows:

    Likely increase in the Index = extra energy * Potential for increase in the Index ** 2 / 1000

with respect to the following constraints.

* in total there are 50000 zillion DSML available for allocation
* no galaxy should be allocated more than 100 zillion DSML or less than 0 zillion DSML
* galaxies with low existence expectancy index below 0.7 should be allocated at least 10% of the total energy available

In [255]:
def total_likely_increase(opt_pred, p_sqr):
    return sum((opt_pred * p_sqr**2) / 1000)


def optimize(data_frame, alpha_low=10, alpha_high=1, steps=100, max_proximity=3):
    max_energy = 50000
    galaxy_lower_bound = 0
    galaxy_upper_bound = 100
    low_end_upper_eei = 0.7
    low_end_galaxy_min_energy = 5000
    
    totals = [0]
    low_end_energies = [0]
#     data_frame['opt_pred'] = max_energy / data_frame.shape[0] - 1
    data_frame['opt_pred'] = np.random.uniform(0,0.1,size=data_frame.shape[0])
    
    low_end = data_frame.loc[data_frame['eei'] < low_end_upper_eei].copy()
    high_end = data_frame.loc[data_frame['eei'] >= low_end_upper_eei].copy()
    
#     dLE = low_end['p_increase2'] / 1000
#     dHE = high_end['p_increase2'] / 1000
    
    dLE = (1/1000)*(1/low_end['opt_pred'] + np.log(low_end['p_increase2']))
    dHE = (1/1000)*(1/high_end['opt_pred'] + np.log(high_end['p_increase2']))
    
    for i in range(steps):
        low_end_energy = low_end['opt_pred'].sum()
        high_end_energy = high_end['opt_pred'].sum()
        total_energy = low_end_energy + high_end_energy
        
        if total_energy>(max_energy - max_proximity):
            print('Stopping... Total Iterations:', i)
            break
            
        if low_end_energy == low_end_energies[-1]:
            print('Stopping... Total Iterations:', i)
            break
        else:
            low_end_energies.append(low_end_energy)
        
        
        low_end['opt_pred'] = (low_end['opt_pred'] + alpha_low*np.exp(dLE)).clip(0,100)
        high_end['opt_pred'] = (high_end['opt_pred'] + alpha_high*np.exp(dHE)).clip(0,100) 
        
        low_end_increase = total_likely_increase(low_end['opt_pred'], low_end['p_increase2'])
        high_end_increase = total_likely_increase(high_end['opt_pred'], high_end['p_increase2'])
        total_increase = low_end_increase + high_end_increase
        
        
        print(f"Sum of Likely Increase: {total_increase:.3f}\tLow end energy: {low_end_energy:.3f}\t"
              f"Total energy: {total_energy:.3f}")
        totals.append(total_increase)
    
    resulting_df = low_end.append(high_end, ignore_index=False)
    resulting_df.reset_index(drop=True, inplace=True)
    totals.remove(0)
    print(f"Max value: {max(totals):.3f}")
    return totals, resulting_df


In [280]:
sums, res_df = optimize(preds, alpha_low=1.6, alpha_high=0.9, steps=5000)

Sum of Likely Increase: 1035.176	Low end energy: 2.660	Total energy: 43.459
Sum of Likely Increase: 1956.457	Low end energy: 119.294	Total energy: 1086.216
Sum of Likely Increase: 2874.775	Low end energy: 235.928	Total energy: 2028.973
Sum of Likely Increase: 3766.256	Low end energy: 352.562	Total energy: 2968.303
Sum of Likely Increase: 4657.737	Low end energy: 469.197	Total energy: 3876.585
Sum of Likely Increase: 5549.218	Low end energy: 585.831	Total energy: 4784.867
Sum of Likely Increase: 6440.699	Low end energy: 702.465	Total energy: 5693.148
Sum of Likely Increase: 7332.180	Low end energy: 819.099	Total energy: 6601.430
Sum of Likely Increase: 8223.661	Low end energy: 935.734	Total energy: 7509.712
Sum of Likely Increase: 9115.142	Low end energy: 1052.368	Total energy: 8417.993
Sum of Likely Increase: 10006.623	Low end energy: 1169.002	Total energy: 9326.275
Sum of Likely Increase: 10898.104	Low end energy: 1285.636	Total energy: 10234.556
Sum of Likely Increase: 11789.585	Low 

In [281]:
sums = pd.DataFrame(sums, columns=['sum_likely_increase'])
px.scatter(sums,
           x=sums.index,
           y='sum_likely_increase')

In [252]:
res_df.opt_pred.sum()

50004.089398296026

In [147]:
res_df.drop(['eei', 'p_increase', 'p_increase2'], axis=1, inplace=True)
res_df.rename(columns= {
    "Index": "index",
    "y_pred": "pred"
}, inplace=True)
res_df

,index,pred,opt_pred
0,7,0.052085,95.459945
1,8,0.041013,98.244556
2,12,0.040515,98.385967
3,27,0.040134,98.495224
4,36,0.041331,98.154990
...,...,...,...
885,885,0.040630,52.301541
886,886,0.053313,52.512606
887,887,0.062798,52.640326
888,888,0.040166,52.292684


In [148]:
res_df.to_csv('../../submissions/pending/lgb_val0525.csv', index=False)

In [282]:
from scipy.optimize import minimize

In [354]:
preds['is_low'] = False
preds.loc[preds.eei < 0.7, 'is_low'] = True
preds.sort_values(by='is_low', ascending=False, inplace=True)
preds.reset_index(drop=True, inplace=True)
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2,is_low
0,216,0.041112,0.073707,0.611594,5.973727,35.685410,True
1,538,0.039145,0.027858,0.589654,6.012981,36.155938,True
2,625,0.040108,0.075984,0.588274,5.993577,35.922961,True
3,779,0.037000,0.003654,0.671839,6.057610,36.694633,True
4,546,0.048877,0.010980,0.659167,5.832300,34.015726,True
...,...,...,...,...,...,...,...
885,304,0.174476,0.033861,1.079911,4.690235,21.998309,False
886,421,0.044880,0.027015,0.925190,5.902608,34.840786,False
887,420,0.139090,0.077212,1.054838,4.903206,24.041429,False
888,295,0.040151,0.003364,0.714065,5.992707,35.912538,False


In [355]:
def neg_total_likely_increase(opt_pred, p_sqr):
    return -sum((opt_pred * p_sqr**2) / 1000)


def objective(energy, p2):
    return neg_total_likely_increase(energy, p2)

In [357]:
m = test.shape[0]
bounds = [(0,100) for _ in range(m)]

# low_p2 = preds.loc[preds['eei'] < 0.7,'p_increase2'].copy().tolist()
# high_p2 = preds.loc[preds['eei'] >= 0.7,'p_increase2'].copy().tolist()
# m_low = len(low_p2)
p2 = preds['p_increase2']

cons = (
    {"type": "ineq", "fun": lambda x: 50000 - sum(x)},
    {"type": "ineq", "fun": lambda x: sum(x[:m_low]) - 5000}
)

history = minimize(objective, x0=np.zeros(m),
                   args=(low_p2, high_p2),
                   bounds=bounds, constraints=cons)

In [358]:
print(sum(history.x))
history.x

50000.000012272554


array([1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 1.00000000e+02,
      

In [360]:
total_likely_increase(history.x, p2)

59353.876870683845

In [364]:
preds['opt_pred'] = history.x
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2,is_low
0,216,0.041112,1.000000e+02,0.611594,5.973727,35.685410,True
1,538,0.039145,1.000000e+02,0.589654,6.012981,36.155938,True
2,625,0.040108,1.000000e+02,0.588274,5.993577,35.922961,True
3,779,0.037000,1.000000e+02,0.671839,6.057610,36.694633,True
4,546,0.048877,1.000000e+02,0.659167,5.832300,34.015726,True
...,...,...,...,...,...,...,...
885,304,0.174476,1.169913e-08,1.079911,4.690235,21.998309,False
886,421,0.044880,1.000000e+02,0.925190,5.902608,34.840786,False
887,420,0.139090,1.427224e-08,1.054838,4.903206,24.041429,False
888,295,0.040151,1.000000e+02,0.714065,5.992707,35.912538,False
